In [65]:
# Load libraries
import numpy as np
import pandas as pd
import io

#  Mock Midterm Exercise

In this exercise you will have to implement code in the sections inside *Fill with Your Code* (*Load Data*, *Data Preprocessing* and *Create Model*) create a model to predict the column *exitus* in the dataset given. The dataset is already split into train, validation, and test subsets. To see to which subset belong each observation, you need to check the *dataset* column.

The code that is already written in this notebook **CANNOT BE CHANGED**. You can only add code in the *Fill with Your Code* section.

You must achieve in the last cell of this notebook an **AUC over test of at least 0.93**.

# Fill With Your Code

### [1] Load Data

In [121]:
dat = pd.read_csv('../data/healthcare_missing.csv', sep = ",")

In [91]:
dat

,date,severity,mortality_ratio,age,num_proc,ambulatory,origin,expected_length,tip_grd,tip_adm,exitus
0,2016-01,NaN,0.408730,12596.0,21.0,0.0,NaN,151.0,Q,1.0,0
1,2016-01,NaN,0.306931,20973.0,22.0,NaN,NaN,99.0,Q,1.0,0
2,2016-01,4.0,0.278481,19611.0,19.0,NaN,NaN,87.0,NaN,1.0,0
3,2016-01,3.0,0.150289,13583.0,22.0,NaN,NaN,100.0,Q,NaN,0
4,2016-01,1.0,0.016573,18042.0,2.0,NaN,NaN,44.0,Q,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...
32701,2016-12,2.0,0.028365,23619.0,2.0,NaN,NaN,2.0,NaN,1.0,0
32702,2016-12,1.0,0.000606,3935.0,1.0,NaN,1.0,2.0,M,1.0,0
32703,2016-12,NaN,0.040452,30163.0,4.0,NaN,NaN,2.0,M,NaN,0
32704,2016-12,NaN,0.000000,29012.0,4.0,NaN,NaN,0.0,NaN,1.0,0


In [122]:
y = dat['exitus']
X = dat.drop(['exitus', 'date'], axis = 1)
X

,severity,mortality_ratio,age,num_proc,ambulatory,origin,expected_length,tip_grd,tip_adm
0,NaN,0.408730,12596.0,21.0,0.0,NaN,151.0,Q,1.0
1,NaN,0.306931,20973.0,22.0,NaN,NaN,99.0,Q,1.0
2,4.0,0.278481,19611.0,19.0,NaN,NaN,87.0,NaN,1.0
3,3.0,0.150289,13583.0,22.0,NaN,NaN,100.0,Q,NaN
4,1.0,0.016573,18042.0,2.0,NaN,NaN,44.0,Q,1.0
...,...,...,...,...,...,...,...,...,...
32701,2.0,0.028365,23619.0,2.0,NaN,NaN,2.0,NaN,1.0
32702,1.0,0.000606,3935.0,1.0,NaN,1.0,2.0,M,1.0
32703,NaN,0.040452,30163.0,4.0,NaN,NaN,2.0,M,NaN
32704,NaN,0.000000,29012.0,4.0,NaN,NaN,0.0,NaN,1.0


### [2]  Data Preprocessing

In [123]:
from sklearn.preprocessing import OneHotEncoder

categorical_vars = ['severity', 'origin', 'ambulatory', 'tip_grd', 'tip_adm']
non_categorical_vars = list(set(X.columns) - set(categorical_vars))
print(categorical_vars)
print(non_categorical_vars)

['severity', 'origin', 'ambulatory', 'tip_grd', 'tip_adm']
['num_proc', 'expected_length', 'mortality_ratio', 'age']


In [124]:
# 3 defining the model
ohe = OneHotEncoder(sparse_output = False)

# 4) Training model
ohe.fit(X)

# 5) Predicting
dat_ohe = pd.DataFrame(ohe.fit_transform(X[categorical_vars]))

# Optional
dat_ohe.columns = ohe.get_feature_names_out()

# Combine numerical and categorical
X_new = pd.concat((X[non_categorical_vars], dat_ohe), axis=1)

X_new

,num_proc,expected_length,mortality_ratio,age,severity_1.0,severity_2.0,severity_3.0,severity_4.0,severity_nan,origin_1.0,...,ambulatory_0.0,ambulatory_1.0,ambulatory_nan,tip_grd_M,tip_grd_Q,tip_grd_nan,tip_adm_1.0,tip_adm_2.0,tip_adm_3.0,tip_adm_nan
0,21.0,151.0,0.408730,12596.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,22.0,99.0,0.306931,20973.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,19.0,87.0,0.278481,19611.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,22.0,100.0,0.150289,13583.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,2.0,44.0,0.016573,18042.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32701,2.0,2.0,0.028365,23619.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
32702,1.0,2.0,0.000606,3935.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
32703,4.0,2.0,0.040452,30163.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
32704,4.0,0.0,0.000000,29012.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [125]:
X_new.isnull().any()

num_proc            True
expected_length     True
mortality_ratio     True
age                 True
severity_1.0       False
severity_2.0       False
severity_3.0       False
severity_4.0       False
severity_nan       False
origin_1.0         False
origin_2.0         False
origin_3.0         False
origin_4.0         False
origin_6.0         False
origin_8.0         False
origin_9.0         False
origin_nan         False
ambulatory_0.0     False
ambulatory_1.0     False
ambulatory_nan     False
tip_grd_M          False
tip_grd_Q          False
tip_grd_nan        False
tip_adm_1.0        False
tip_adm_2.0        False
tip_adm_3.0        False
tip_adm_nan        False
dtype: bool

In [126]:
### Fill missing values
from fancyimpute import IterativeImputer as MICE # pip install fancyimpute
numerical_vars = list(set(X_new.columns) - {'exitus', 'dataset'})
model = MICE()
X_new[numerical_vars] = model.fit_transform(X_new[numerical_vars])

### [3] Create Model

In [128]:
from sklearn.model_selection import train_test_split

perc_values = [0.7, 0.15, 0.15]
X_train, X_valtest, y_train, y_valtest = train_test_split(X_new, y, stratify = y, test_size=perc_values[1] + perc_values[2], random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, stratify = y_valtest, test_size= perc_values[2] / (perc_values[1] + perc_values[2]), random_state=1)

In [131]:
from sklearn.ensemble import RandomForestClassifier as model_constructor
from sklearn.metrics import roc_auc_score as metric

# Random Forest
n_estimators_values = [10, 100, 1000]
max_features_values = [2, 5, 10]
max_samples_values = [100, 1000, X_train.shape[0]]

params_grid = {'max_features': max_features_values,
              'n_estimators': n_estimators_values,
               'max_samples': max_samples_values}

num_iter = 1;
grid_results = pd.DataFrame(columns = ('max_features',
                                       'n_estimators',
                                       'max_samples',
                                       'metric_train',
                                       'metric_val'))

for max_features in params_grid['max_features']:
    for n_estimators in params_grid['n_estimators']:
        for max_samples in params_grid['max_samples']:

                        # Print trace
                        print('Iteracion = ' + str(num_iter))

                        # [3] Define model
                        model = model_constructor(max_features = max_features,
                                                  n_estimators = n_estimators,
                                                  max_samples = max_samples,
                                                  random_state = 0)

                        # [4] Train model
                        model.fit(X_train, y_train)


                        # [5] Predict
                        pred_train = model.predict(X_train) # predict!
                        pred_val = model.predict(X_val) # predict!

                        # [6] Compute metric
                        metric_train = metric(y_train, pred_train)
                        metric_val = metric(y_val, pred_val)

                        # print error
                        print('Metric train = %.2f - Metric validation = %.2f.'
                              % (metric_train, metric_val))

                        # Save iteration results
                        grid_results.loc[num_iter]=[ max_features,
                                                    n_estimators,
                                                    max_samples,
                                                 metric_train,
                                                 metric_val]
                        num_iter += 1




Iteracion = 1
Metric train = 0.51 - Metric validation = 0.51.
Iteracion = 2
Metric train = 0.54 - Metric validation = 0.54.
Iteracion = 3
Metric train = 0.93 - Metric validation = 0.59.
Iteracion = 4
Metric train = 0.50 - Metric validation = 0.50.
Iteracion = 5
Metric train = 0.51 - Metric validation = 0.52.
Iteracion = 6
Metric train = 1.00 - Metric validation = 0.59.
Iteracion = 7
Metric train = 0.50 - Metric validation = 0.50.
Iteracion = 8
Metric train = 0.51 - Metric validation = 0.51.
Iteracion = 9
Metric train = 1.00 - Metric validation = 0.58.
Iteracion = 10
Metric train = 0.51 - Metric validation = 0.51.
Iteracion = 11
Metric train = 0.55 - Metric validation = 0.55.
Iteracion = 12
Metric train = 0.94 - Metric validation = 0.59.
Iteracion = 13
Metric train = 0.50 - Metric validation = 0.50.
Iteracion = 14
Metric train = 0.53 - Metric validation = 0.54.
Iteracion = 15
Metric train = 1.00 - Metric validation = 0.59.
Iteracion = 16
Metric train = 0.50 - Metric validation = 0.50.
I

In [135]:
grid_results = grid_results.sort_values(by = ['metric_val', 'metric_train'], ascending = [False, True])
best_model = grid_results.iloc[0]
best_model

max_features       10.000000
n_estimators       10.000000
max_samples     22894.000000
metric_train        0.936792
metric_val          0.606499
Name: 21, dtype: float64

In [136]:
print('Old train data size = ' + str(X_train.shape))
print('Old train target size = ' + str(y_train.shape))

# Combine train and validación
X_train = np.concatenate((X_train, X_val), axis = 0)
y_train = np.concatenate((y_train, y_val), axis = 0)

print('New train data size = ' + str(X_train.shape))
print('New train target size = ' + str(y_train.shape))

Old train data size = (22894, 27)
Old train target size = (22894,)
New train data size = (27800, 27)
New train target size = (27800,)


In [138]:
# [3] Define model
model = model_constructor(criterion = 'gini',
                          max_depth = None,
                          min_samples_split = 2,
                          min_samples_leaf = 1,
                          max_features = int(best_model.max_features),
                          n_estimators =  int(best_model.n_estimators),
                          max_samples = int(best_model.max_samples),
                          random_state = 0) # Use same random_state as in training!!!

# Evaluate Model

In [140]:
# [4] Train model
# [4] Train model
model.fit(X_train, y_train)

# [5] Predict
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

# [6] Compute metric
metric_train = metric(y_train, pred_train)
metric_test = metric(y_test, pred_test)

J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [141]:
# print error
print('Metric train = %.2f - Metric val = %.2f - Metric test = %.2f'
      % (metric_train, metric_val, metric_test))

Metric train = 0.90 - Metric val = 0.60 - Metric test = 0.58
